In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import pandas as pd

HashData=[]
TimeData=[]
From=[]
To=[]
Value=[]
TimeStamp=[]
TransactionFee=[]
GasPrice=[]
Block=[]



opt=webdriver.ChromeOptions()
opt.add_argument("--headless")

s=Service("C:\Program Files (x86)\chromedriver.exe")
driver= webdriver.Chrome(service=s )

Start_Block=input("ENTER STARTING BLOCK NUMBER ")  #Block_Number="14800000"

TotalBlocks =int(input("Total Blocks To Fetch"))

for i in range(TotalBlocks):

    Block_Number=str(i+int(Start_Block))
    
    Block_Page_url= "https://etherscan.io/block/"+ Block_Number

    Txs_Page_url="https://etherscan.io/txs?block="+ Block_Number

    driver.get(Txs_Page_url)

    Total_txs_Info = driver.find_element(By.XPATH,"/html/body/div[1]/main/div[3]/div/div/div[2]/p/span")

    Total_txs= int(Total_txs_Info.text.split()[3])

    print("Total transaction in the block" , Total_txs)

    Pages= int(Total_txs/50)+1 if Total_txs%50 else int(Total_txs/50)

    print("Total Pages" , Pages)

    Current_Page_Txs = driver.find_elements(By.CLASS_NAME,"myFnExpandBox_searchVal")

    MissingVal=0   
    for x in Current_Page_Txs:
        Hash=x.text 
        Txs_url="https://etherscan.io/tx/"+ Hash
        driver.execute_script("window.open()")
        driver.switch_to.window(driver.window_handles[1])
        driver.get(Txs_url)
        try:
            time_element=driver.find_element(By.XPATH,"/html/body/div[1]/main/div[3]/div[1]/div[2]/div[1]/div/div[4]/div/div[2]/span[2]").text
        except :
            MissingVal=MissingVal+1
        else:
            HashData.append(Hash)
            To.append(driver.find_element(By.ID,"contractCopy").text)
            From.append(driver.find_element(By.ID,"addressCopy").text)
            Value.append(driver.find_element(By.ID,"ContentPlaceHolder1_spanValue").text)
            TimeStamp.append(driver.find_element(By.XPATH,"/html/body/div[1]/main/div[3]/div[1]/div[2]/div[1]/div/div[4]/div/div[2]").text)
            TransactionFee.append(driver.find_element(By.ID,"ContentPlaceHolder1_spanTxFee").text)
            GasPrice.append(driver.find_element(By.ID,"ContentPlaceHolder1_spanGasPrice").text)
            TimeData.append(time_element)
                
        print(HashData[-1]+"  "+TimeData[-1]+"  "+To[-1]+"  "+From[-1]+"  "+Value[-1]+"  "+TimeStamp[-1]+"  "+TransactionFee[-1]+"  "+GasPrice[-1])
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(2) 


    for i in range(Pages-1):
        driver.get(Txs_Page_url+"&p="+str(i+2))

        Current_Page_Txs = driver.find_elements(By.CLASS_NAME,"myFnExpandBox_searchVal")

        for x in Current_Page_Txs:
            Hash=x.text    
            Txs_url="https://etherscan.io/tx/"+ Hash
            driver.execute_script("window.open()")
            driver.switch_to.window(driver.window_handles[1])
            driver.get(Txs_url)
            try:
                time_element=driver.find_element(By.XPATH,"/html/body/div[1]/main/div[3]/div[1]/div[2]/div[1]/div/div[4]/div/div[2]/span[2]").text
            except:
                MissingVal=MissingVal+1
            else:
                HashData.append(Hash)
                To.append(driver.find_element(By.ID,"contractCopy").text)
                From.append(driver.find_element(By.ID,"addressCopy").text)
                Value.append(driver.find_element(By.ID,"ContentPlaceHolder1_spanValue").text)
                TimeStamp.append(driver.find_element(By.XPATH,"/html/body/div[1]/main/div[3]/div[1]/div[2]/div[1]/div/div[4]/div/div[2]").text)
                TransactionFee.append(driver.find_element(By.ID,"ContentPlaceHolder1_spanTxFee").text)
                GasPrice.append(driver.find_element(By.ID,"ContentPlaceHolder1_spanGasPrice").text)
                TimeData.append(time_element)
            
            print(HashData[-1]+"  "+TimeData[-1]+"  "+To[-1]+"  "+From[-1]+"  "+Value[-1]+"  "+TimeStamp[-1]+"  "+TransactionFee[-1]+"  "+GasPrice[-1])
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            time.sleep(2) 

        
data_tuples = list(zip(HashData[1:],TimeData[1:],To[1:],From[1:],Value[1:],TimeStamp[1:],TransactionFee[1:],GasPrice[1:]))
df = pd.DataFrame(data_tuples, columns=['Hash','CompletionTime','To','From','Value','TimeStamp','TxnFee','GasPrice'])
df.to_csv('EthDataCSV',index=False)

driver.quit( )    


